Notebook to test OTP configuration.
Written by Vivek
Started July 7th, 2016

We need to test the different parameters like bike and walk, etc.  Build functions that test numerous locations, settings, and generate times.

Current Todos:

TODO: Geocode Addresses into GPS using Google

TODO: Push resulting data into pandas dataframe

In [14]:
from datetime import datetime
import requests
import json
import pandas as pd
import numpy as np

Sample Api Call:
http://vta2.amigocloud.com/api/otp/plan?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false

http://vta2.amigocloud.com/api/otp/plan?mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.4002813%2C-121.9396906&toPlace=37.326763%2C-121.856533&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=15&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE

In [127]:
def print_transit(query_json):
    for a in query_json['plan']['itineraries']:
        print('start: '+ tclean(a['startTime']))
        print('end: '+ tclean(a['endTime']))
        print('duration: '+ str(a['duration']/60) + ' minutes')
        print('waitingTime: '+ str(a['waitingTime']/60) + ' minutes')
        print('transitTime: '+ str(a['transitTime']/60) + ' minutes')
        print('walkDistance: '+ str(a['walkDistance']/1000*.621371) + ' miles')
        print('transfers: '+ str(a['transfers']))
        for each in a['legs']:
            print('   ' + each['from']['name'])
            print('   StartTime: '+ tclean(each['startTime']) + " mode: " + each['mode'])
            print("   headsign: " + str(each.get('headsign')))
            print('   endTime:   ' + tclean(each['endTime']) + " duration: " + str(each['duration']/60))    

In [434]:
rand_starts=pd.read_csv('Rand_Starts.csv')
rand_dests=pd.read_csv('Rand_Dests.csv')
known_starts=pd.read_csv('Known_Starts.csv')
known_dests=pd.read_csv('Known_Dests.csv')

df = pd.DataFrame();

#df['start_lat'] = rand_starts['Lat']
#df['start_long'] = rand_starts['Long']
#df['dest_lat'] = rand_dests['Lat']
#df['dest_long'] = rand_dests['Long']

df['start_lat'] = known_starts['Lat']
df['start_long'] = known_starts['Long']
df['start_loc']= known_starts['Desc']
df['dest_lat'] = known_dests['Lat']
df['dest_long'] = known_dests['Long']
df['dest_loc'] = known_dests['Desc']

In [435]:
def tclean(dto):
    return str(datetime.fromtimestamp(dto/1000))

In [437]:
df.head()
#Random From and To pints

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose
2,37.400787,-121.868619,1684 Garvey Pl,37.400281,-121.939691,3331 N. 1st St VTA
3,37.400281,-121.939691,3331 N. 1st St VTA,37.333539,-121.885089,"116 Paseo De San Antonio Walk, San Jose, CA 95..."
4,37.397893,-121.868478,"2755 Lucena Dr, San Jose, CA 95132, USA",37.400281,-121.939691,3331 N. 1st St VTA


In [238]:
#https://github.com/opentripplanner/OpenTripPlanner/blob/257ac13150b40942fdae4812007545a66546d7da/src/main/java/org/opentripplanner/routing/core/RoutingRequest.java
row = 3
payload = {}
payload.update({'fromPlace': '%s,%s' % (df['start_lat'][row],df['start_long'][row])})
payload.update({'toPlace': '%s, %s' % (df['dest_lat'][row], + df['dest_long'][row])})
payload.update({'mode':'WALK,BUSISH,TRAINISH'})
payload.update({'date':'2016-07-14'})
payload.update({'time':'07:00'})
payload.update({'maxTransfers':3,'optimize':'QUICK'})
payload.update({'numItineraries':5})
payload.update({'maxWalkDistance':5000})
payload.update({'bikeSpeed':5})
payload.update({'arriveBy':'false'})
payload.update({'waitAtBeginning':5.95})
payload.update({'walkReluctance':12})
payload.update({'maxPreTransitTime':100})

r = requests.get('http://vta2.amigocloud.com/api/otp/plan?', params=payload)
#r.text
print_transit(r.json())

start: 2016-07-14 07:00:29
end: 2016-07-14 07:32:11
duration: 31.7 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 24.666666666666668 minutes
walkDistance: 0.32753311969287974 miles
transfers: 0
   parking aisle
   StartTime: 2016-07-14 07:00:29 mode: WALK
   headsign: None
   endTime:   2016-07-14 07:04:49 duration: 4.333333333333333
   RIVER OAKS STATION (1)
   StartTime: 2016-07-14 07:04:50 mode: TRAM
   headsign: SANTA TERESA
   endTime:   2016-07-14 07:29:30 duration: 24.666666666666668
   SAN ANTONIO STATION (1)
   StartTime: 2016-07-14 07:29:31 mode: WALK
   headsign: None
   endTime:   2016-07-14 07:32:11 duration: 2.6666666666666665
start: 2016-07-14 07:10:29
end: 2016-07-14 07:42:11
duration: 31.7 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 24.666666666666668 minutes
walkDistance: 0.32753311969287974 miles
transfers: 0
   parking aisle
   StartTime: 2016-07-14 07:10:29 mode: WALK
   headsign: None
   endTime:   2016-07-14 07:14:49 duration: 4.3

In [443]:
#for each in range(1,59):
    #print(each)
row = 7
payload = {}

#walkReluctance=[0,10,20,100]
#time = [1,2,3,4,5]
#maxWalk
payload.update({'date':'2016-07-14'})
payload.update({'time':'07:00'})
payload.update({'maxTransfers':3,'optimize':'QUICK'})
payload.update({'fromPlace': '%s,%s' % (df['start_lat'][row],df['start_long'][row])})
payload.update({'toPlace': '%s, %s' % (df['dest_lat'][row], + df['dest_long'][row])})
#https://github.com/opentripplanner/OpenTripPlanner/blob/257ac13150b40942fdae4812007545a66546d7da/src/main/java/org/opentripplanner/routing/core/TraverseModeSet.java
#payload.update({'mode':'WALK,TRAINISH'})
#payload.update({'mode':'WALK,TRAM'})
#payload.update({'mode':'WALK,TRAINISH,RAIL,CABLE_CAR'})
#payload.update({'mode':'WALK,TRAM'})
#payload.update({'mode':'WALK,TRAM, BUSISH'})
payload.update({'mode':'WALK,CAR,TRANSIT'})
#payload.update({'walkReluctance':20})
payload.update({'walkReluctance':20})
#payload.update({'waitReluctance':0})
payload.update({'time':'07:00'})
#payload.update({'time':'07:%s' % (each)})
#payload.update({'walkBoardCost':500})
#payload.update({'bikeBoardCost':0})
payload.update({'maxTranfers':2})
#payload.update({'preTransitPenalty':})
#payload.update({'waitAtBeginning':100000000})
#payload.update({'maxPreTransitTime':10000})
payload.update({'maxWalkDistance':5000})
payload.update({'numItineraries':5})
#payload.update({'softWalkPenalty':60.0})
#payload.update({'softWalkOverageRate':5.0})
r = requests.get('http://vta2.amigocloud.com/api/otp/plan?', params=payload)
#r.text
print_transit(r.json())

start: 2016-07-14 07:12:07
end: 2016-07-14 08:02:41
duration: 50.56666666666667 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 40.166666666666664 minutes
walkDistance: 0.232350080293793 miles
transfers: 0
   Start point 0.0430995197605577
   StartTime: 2016-07-14 07:12:07 mode: CAR
   headsign: None
   endTime:   2016-07-14 07:17:30 duration: 5.383333333333334
   corner of path and service road
   StartTime: 2016-07-14 07:17:30 mode: WALK
   headsign: None
   endTime:   2016-07-14 07:17:59 duration: 0.48333333333333334
   OHLONE-CHYNOWETH STATION (0)
   StartTime: 2016-07-14 07:18:00 mode: TRAM
   headsign: ALUM ROCK
   endTime:   2016-07-14 07:58:10 duration: 40.166666666666664
   RIVER OAKS STATION (0)
   StartTime: 2016-07-14 07:58:11 mode: WALK
   headsign: None
   endTime:   2016-07-14 08:02:41 duration: 4.5
start: 2016-07-14 07:25:07
end: 2016-07-14 08:11:41
duration: 46.56666666666667 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 36.166666666666664

In [415]:
r.url

'http://vta2.amigocloud.com/api/otp/plan?date=2016-07-14&time=07%3A00&numItineraries=5&maxWalkDistance=5000&maxTransfers=3&maxTranfers=2&fromPlace=37.3966147%2C-121.8682322&optimize=QUICK&toPlace=37.400281%2C+-121.939691&walkReluctance=20&mode=WALK%2CTRANSIT'

In [197]:
r.url

'http://vta2.amigocloud.com/api/otp/plan?maxPreTransitTime=100&time=07%3A00&maxTransfers=3&arriveBy=false&maxWalkDistance=5000&waitAtBeginning=12&bikeSpeed=5&mode=TRAINISH%2CWALK&date=2016-07-14&toPlace=37.3335387%2C+-121.8850891&numItineraries=5&preTransitPenalty=100&maxTranfers=2&fromPlace=37.400281%2C-121.939691&optimize=QUICK&walkReluctance=20'

In [192]:
df.head()
#Random From and To pints

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose
2,37.400787,-121.868619,1684 Garvey Pl,37.400281,-121.939691,3331 N. 1st St VTA
3,37.400281,-121.939691,3331 N. 1st St VTA,37.333539,-121.885089,"116 Paseo De San Antonio Walk, San Jose, CA 95..."
4,37.397893,-121.868478,"2755 Lucena Dr, San Jose, CA 95132, USA",37.400281,-121.939691,3331 N. 1st St VTA


In [ ]:
, '.join([str(x) for x in list])
''.join(map(str,list))

In [364]:
walkReluctance=[0,10,20,100]
time = [1,2,3,4,5]
cartesian((walkReluctance,time))


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:47: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:49: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


array([[  0,   1],
       [  0,   2],
       [  0,   3],
       [  0,   4],
       [  0,   5],
       [ 10,   1],
       [ 10,   2],
       [ 10,   3],
       [ 10,   4],
       [ 10,   5],
       [ 20,   1],
       [ 20,   2],
       [ 20,   3],
       [ 20,   4],
       [ 20,   5],
       [100,   1],
       [100,   2],
       [100,   3],
       [100,   4],
       [100,   5]])

In [365]:
import numpy as np

def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.

    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.

    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.

    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])

    """
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

walkReluctance=[0,10,20,100]
time = [1,2,3,4,5]
waitAtBeginning = [1,10,50,100,1000,2000,5000,10000,100000]
array = cartesian((walkReluctance,time,waitAtBeginning))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:47: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:49: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:48: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:50: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


numpy.ndarray